# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [47]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
from IPython.display import IFrame
import folium
from folium.plugins import MarkerCluster
import geoviews as gv
import holoviews as hv


# Import API key
from api_keys import geoapify_key



In [103]:
geoapify_key = "31f7eaa68819402cb68089fcc407e8ca"
base_url = "https://api.geoapify.com/v2/places"



In [104]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

file_path = "/Users/owner/Library/CloudStorage/OneDrive-Personal/WeatherPy/output_data/cities.csv"

city_data_df = pd.read_csv(file_path)



# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,puerto ayora,-0.7393,-90.3518,297.29,87,35,3.30,EC,1702257885
1,1,gagliano del capo,39.8442,18.3695,282.83,93,89,5.66,IT,1702257885
2,2,pirane,-25.7324,-59.1088,293.77,83,0,1.86,AR,1702257885
3,3,sihanoukville,10.6093,103.5296,300.12,89,40,2.06,KH,1702257885
4,4,longyearbyen,78.2186,15.6401,273.06,74,75,5.66,SJ,1702257784


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [105]:
%%capture --no-display

# Configure the map plot
map_center = [city_data_df["Lat"].iloc[0], city_data_df["Lng"].iloc[0]]
m = folium.Map(location=[0, 0], zoom_start=2)

marker_cluster = MarkerCluster().add_to(m)


# Loop through each city in the DataFrame
for index, row in city_data_df.iterrows():
    # Create a circle marker with the size based on humidity
    folium.CircleMarker(
        location=[row["Lat"], row["Lng"]],
        radius=row["Humidity"] / 5,  
        popup=f"City: {row['City']}<br>Humidity: {row['Humidity']}%",
        fill=True,
        color="blue",
        fill_color="blue",
        fill_opacity=0.7
    ).add_to(marker_cluster)

# Save the map as an HTML file
m.save("output_data/city_map.html")


# Display the map
IFrame("output_data/city_map.html", width=800, height=600)

In [106]:
# Define a function to create a GeoDataFrame
def create_gdf(df):
    return gv.Points(df, ['Lng', 'Lat']).opts(
        color='Humidity', cmap='viridis', size=5,
        tools=['hover'], title='City Humidity Map'
    )

# Create the GeoDataFrame
city_gdf = create_gdf(city_data_df)

# Display the map for all cities
city_gdf.opts(width=800, height=600)

:Points   [Lng,Lat]   (City_ID,City,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [107]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_conditions = (
    (city_data_df["Max Temp"].between(200, 300)) &
    (city_data_df["Humidity"].between(20, 70)) &
    (city_data_df["Cloudiness"] < 40) &
    (city_data_df["Wind Speed"] < 15)
)

# Apply conditions to create ideal_cities_df
ideal_cities_df = city_data_df[ideal_conditions]

# Drop any rows with null values in ideal_cities_df
ideal_cities_df = ideal_cities_df.dropna()

# Display sample data for ideal_cities_df
print("Number of cities in ideal_cities_df:", len(ideal_cities_df))
ideal_cities_df.head()


Number of cities in ideal_cities_df: 74


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
6,6,blackmans bay,-43.0167,147.3167,295.66,52,8,0.89,AU,1702257885
8,8,naraina,26.7833,75.2000,285.98,35,0,1.53,IN,1702257885
10,10,tidjikja,18.5564,-11.4271,292.61,24,4,6.63,MR,1702257761
11,11,tura,25.5198,90.2201,287.14,62,0,1.43,IN,1702257771
12,12,waitangi,-43.9535,-176.5597,284.79,59,0,2.24,NZ,1702257755


### Step 3: Create a new DataFrame called `hotel_df`.

In [108]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_cities_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name"
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df.head()


,City,Country,Lat,Lng,Humidity,Hotel Name
6,blackmans bay,AU,-43.0167,147.3167,52,
8,naraina,IN,26.7833,75.2000,35,
10,tidjikja,MR,18.5564,-11.4271,24,
11,tura,IN,25.5198,90.2201,62,
12,waitangi,NZ,-43.9535,-176.5597,59,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [133]:
# Set parameters to search for a hotel
radius = 10000  # Adjust the radius as needed
api_key = '31f7eaa68819402cb68089fcc407e8ca' # the APT kept giving me errors, I generated new API
category = 'hotel'  

params = {
    'apiKey': api_key,
    'categories': category,
    'radius': radius,
    'limit': 1,
    'sort': 'distance'
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # Get latitude and longitude from the DataFrame
    lat, lng = row["Lat"], row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"point({lng},{lat})"
    params["bias"] = f"point({lng},{lat})"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make an API request using the params dictionary
    response = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = response.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
blackmans bay - nearest hotel: No hotel found
naraina - nearest hotel: No hotel found
tidjikja - nearest hotel: No hotel found
tura - nearest hotel: No hotel found
waitangi - nearest hotel: No hotel found
boquira - nearest hotel: No hotel found
tchintabaraden - nearest hotel: No hotel found
pokhara - nearest hotel: No hotel found
tshabong - nearest hotel: No hotel found
invercargill - nearest hotel: No hotel found
margaret river - nearest hotel: No hotel found
poronaysk - nearest hotel: No hotel found
salalah - nearest hotel: No hotel found
konnur - nearest hotel: No hotel found
debark' - nearest hotel: No hotel found
adamstown - nearest hotel: No hotel found
upington - nearest hotel: No hotel found
sheridan - nearest hotel: No hotel found
crane - nearest hotel: No hotel found
ushuaia - nearest hotel: No hotel found
edd - nearest hotel: No hotel found
lompoc - nearest hotel: No hotel found
jiwani - nearest hotel: No hotel found
woodward - nearest hotel: No hotel f

,City,Country,Lat,Lng,Humidity,Hotel Name
6,blackmans bay,AU,-43.0167,147.3167,52,No hotel found
8,naraina,IN,26.7833,75.2000,35,No hotel found
10,tidjikja,MR,18.5564,-11.4271,24,No hotel found
11,tura,IN,25.5198,90.2201,62,No hotel found
12,waitangi,NZ,-43.9535,-176.5597,59,No hotel found
...,...,...,...,...,...,...
527,khamis mushait,SA,18.3064,42.7292,36,No hotel found
541,al burayqah,LY,30.4062,19.5739,60,No hotel found
551,rikaze,CN,29.2500,88.8833,39,No hotel found
554,masterton,NZ,-40.9597,175.6575,39,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [92]:
%%capture --no-display

# Create a Folium map centered at the first city's coordinates
map_center = [hotel_df["Lat"].iloc[0], hotel_df["Lng"].iloc[0]]
hotel_map = folium.Map(location=map_center, zoom_start=10)

# Iterate through the DataFrame to add markers for each city
for index, row in hotel_df.iterrows():
    # Get city coordinates
    city_coords = [row["Lat"], row["Lng"]]
    
    # Create a tooltip/hover message with city, hotel name, and country information
    tooltip = f"{row['City']}, {row['Country']} - Hotel: {row['Hotel Name']}"

    # Add a marker for each city with the tooltip
    folium.Marker(location=city_coords, tooltip=tooltip).add_to(hotel_map)

# Display the map
hotel_map